<a href="https://colab.research.google.com/github/wendychen215/aop113b/blob/main/EX05_04_Reply_Template_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install line-bot-sdk flask pyngrok flask_ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 5.3 MB/s eta 0:00:00


In [ ]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK'))
    ngrok.connect(5000)
    run_with_ngrok(app)


from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage, StickerMessage, ImageMessage, AudioMessage, VideoMessage, LocationMessage,
    TemplateMessage, ConfirmTemplate, MessageAction,
    CarouselTemplate, CarouselColumn, URIAction, PostbackAction,
)

app = Flask(__name__)

@app.route("/", methods = ["GET"])
def index():
    return "<p>Hello!</p>"

configuration = Configuration(access_token=userdata.get('access_token'))
handler = WebhookHandler(userdata.get('channel_secret'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

import google.generativeai as genai

genai.configure(api_key=userdata.get('apikey'))
model = genai.GenerativeModel('gemini-2.0-flash')
def ask_gemini(question):
  response = model.generate_content(question)
  return response.text

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        action = event.message.text
        if action == "confirm":
          template = ConfirmTemplate(
              text = "你喜歡麻辣鍋嗎",
              actions = [
                  MessageAction(label="是", text="是"),
                  MessageAction(label="否", text="否")
              ]
            )
          reply = TemplateMessage(
            alt_text = "這是輪播視窗",
            template = template
          )

        elif action == "carousel":
          carousel_template = CarouselTemplate(
              columns = [
                  CarouselColumn(
                      thumbnail_image_url = "https://upload.wikimedia.org/wikipedia/zh/2/24/Princess_Mononoke_Japanese_Poster_%28Movie%29.jpg",
                      title = "魔法公主",
                      text = "吉卜力工作室製作，由宮崎駿執導，於1997年7月12日首映的日本動畫電影。",
                      actions = [
                          URIAction(label = "查看詳情", uri = "https://zh.wikipedia.org/zh-tw/%E9%AD%94%E6%B3%95%E5%85%AC%E4%B8%BB"),
                          URIAction(label = "查看詳情", uri = "https://www.youtube.com/watch?v=nc3KdY-HkFM")
                      ]
                  )
              ]
          )
          reply = TemplateMessage(
            alt_text = "這是輪播視窗",
            template = carousel_template
          )
        else:
          reply = TextMessage(text = "Please type 'confirm' or 'carousel'")

        line_bot_api.reply_message(
          ReplyMessageRequest(
              reply_token=event.reply_token,
              messages=[
                  reply
                ]
            )
        )



ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://09c6-34-169-106-104.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [13/May/2025 08:01:09] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 08:01:16] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 08:01:21] "POST /callback HTTP/1.1" 200 -
